In [ ]:
!pip install earthengine-api
!pip install geemap

In [2]:
import ee
import geemap

# Authenticating Google Earth Engine
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Rqw09L_IJEKUsnGO4PIjPHELg3EC4RRGFqoQkaTsIrQ&tc=Tftc8k3Novmy4cD5gPOpHEmCYFc0zmhY9oxjY69ij9k&cc=a9BePM2vZ1u0zfnZalPa892ZkKRRRjPTvjNk3vWbJ24

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPeXyduPki_63ka2vtnxE3tW8wopxKsnmK2b7A3mhXnpHC7xEtr-yQ

Successfully saved authorization token.


In [6]:
Map = geemap.Map()

# Defining the geographic center coordinates of Africa and zoom level
lon = 22.576728206442617;
lat = 1.1642709033901901;
zoom = 3

# Defining the desired year
year = 2020

# Defining the duration in months
duration = 12

# Defining the start and end dates
startDate = ee.Date.fromYMD(year, 1, 1)
endDate = startDate.advance(duration, 'month')

# Taking image collections
modImage = ee.ImageCollection('MODIS/006/MOD14A1')
mydImage = ee.ImageCollection('MODIS/006/MYD14A1')

# Clipping the images to Africa boundaries
africa = ee.Geometry.Polygon([
[-16.056859987284096,6.816095979064512],
[-8.498266237284096,2.261330495738635],
[-8.498266237284096,2.261330495738635],
[2.7517337627159044,-18.501647831209322],
[8.728296262715904,-37.03865007848229],
[18.747827512715904,-38.428765115844996],
[33.3376712627159,-34.90499105038275],
[49.1579837627159,-30.77124671124308],
[55.4861087627159,-7.211156095611061],
[54.4314212627159,6.4668932767733915],
[52.1462650127159,13.900847135528284],
[43.5329837627159,11.500316524638372],
[37.9079837627159,20.446416356369017],
[30.525171262715904,33.558651844943036],
[19.978296262715904,33.59526460501117],
[14.473305950148907,33.996977842734935],
[11.792641887648907,36.43794291741861],
[10.430337200148907,38.01247895021939],
[5.552407512648907,37.386611763588405],
[0.322915325148907,36.89618219187306],
[-6.532553424851093,36.11910509840554],
[-22.606373607348633,23.737810071603807],
[-21.375904857348633,14.958292054033121],
[-16.056859987284096,6.816095979064512]])
# Function to clip the image
def clipImage(image):
    return image.clip(africa)

def getFireMask(image):
    # Values above 7 indicate active pixels
    return image.select('FireMask').gte(7)

def getMaxFRP(image):
    # FRP (Fire Radiative Power) multiplied by 0.1 to convert to MW (MegaWatt)
    return image.select('MaxFRP').multiply(0.1)
    
modImageClipped = modImage.map(clipImage)
mydImageClipped = mydImage.map(clipImage)

# Filtering by date range
modImageFiltered = modImageClipped.filterDate(startDate, endDate)
mydImageFiltered = mydImageClipped.filterDate(startDate, endDate)


# Creating masks for MODIS and MYD images
modImageFilteredMask = modImageFiltered.map(getFireMask).sum()
mydImageFilteredMask = mydImageFiltered.map(getFireMask).sum()

# Calculating FRP for MODIS and MYD
modImageFilteredFrp = modImageFiltered.map(getMaxFRP).sum()
mydImageFilteredFrp = mydImageFiltered.map(getMaxFRP).sum()

def getBurnDate(image):
    return image.select('BurnDate')
# Taking the burned area image collection
burnedImage = ee.ImageCollection('MODIS/006/MCD64A1')

# Clipping the burned area image
burnedImageClipped = burnedImage.map(clipImage)


# Filtering by burned area date
burnedImageFiltered = burnedImageClipped.filterDate(startDate, endDate)

# Creating a mask for burned areas
burnedImageFilteredMask = burnedImageFiltered.map(getBurnDate).min()

# Adding layers to the map
Map.addLayer(modImageFilteredMask.selfMask(), {'palette': 'orange'}, 'MOD active fire')
Map.addLayer(mydImageFilteredMask.selfMask(), {'palette': 'red'}, 'MYD active fire')
Map.addLayer(burnedImageFilteredMask.selfMask(), {'palette': 'black'}, 'Burned area')
Map.addLayer(modImageFilteredFrp.selfMask(), {'palette': 'yellow'}, 'MOD frp')
Map.addLayer(mydImageFilteredFrp.selfMask(), {'palette': 'green'}, 'MYD frp')

Map.setCenter(lon, lat, zoom)

Map

Map(center=[1.1642709033901901, 22.576728206442617], controls=(WidgetControl(options=['position', 'transparent…